### Task 2.2 Supervised preprocessing and visualization

- Load dataset (diabetes.csv)
  - Use `pandas.read_csv()`
- Define and apply IQR-based outlier detection
  - Compute IQR (Q3 - Q1) for numerical features
  - Mark values outside [Q1 - 1.5*IQR, Q3 + 1.5*IQR] as outliers
- Replace outliers with NA and select complete cases
  - Use `numpy.nan` to replace outliers
  - Drop rows with missing values using dropna()
  - Consider the meaning of zeros before removal
- [Optional] Describe single features
  - Use pandas.describe() for summary statistics
- Rank features using Chi-squared statistics
  - Use chi2 from sklearn.feature_selection
  - Identify the most and least discriminating features
- Create boxplot and distribution plots
  - Use seaborn.boxplot() for boxplots
  - Use seaborn.histplot() or matplotlib.pyplot.hist() for distribution plots
  - Compare the highest and lowest ranked features
- Update Git repository
  - Run `git add`, `git commit -m "Feature selection and visualization"`, `git push` in the terminal